# 🛳️ Titanic Data Investigation

**Scenario:**
You’ve been hired to prepare clean Titanic passenger data for a museum exhibit.
The original data comes from multiple sources.
Your mission is to:
1. Detect and fix missing information
2. Reshape messy structures into tidy format
3. Combine data for richer analysis
4. Transform and normalize values
5. Create one clear visualization for the exhibit

Every wrangling step you take helps make history more understandable!


## 0. Setup & Reproducibility

In [ ]:
!pip install pandas==2.3.3 seaborn==0.13.2 numpy==2.3.4

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

np.random.seed(42)
pd.__version__, np.__version__, sns.__version__

## 1) Load & Inspect

**Goal:** Understand the dataset’s shape, types, and missingness.

**Do this:**
1. Inspect the `titanic` dataset with `head()`, `info()`
2. Count missing values per column
3. Choose 3 candidate columns to clean


In [ ]:
df = sns.load_dataset("titanic").iloc[:-100]
df.head()


In [ ]:
#SOLUTION
df.info()
df.isna().sum().sort_values(ascending=False).head(10)


<details>
<summary>💡 Hint</summary>

- Use `df.info()` to see dtypes and non-null counts
- Use `df.isna().sum()` to count missing values
- Typical targets here: `deck`, `age`, `embarked`
</details>


### Checkpoint 1

In [ ]:
expected = {"survived","pclass","sex","age","sibsp","parch","fare","embarked","class","who","adult_male","deck","embark_town","alive"}
assert expected.issubset(set(df.columns))
print("✅")


## 2) Handle Missing Data

**Goal:** Handle common observational missing values.

**Do this:**
1. Fill missing `age` with the **median**
2. Fill missing `embarked` with the **mode**
3. Replace missing `deck` with `"Unknown"` (categorical placeholder)
4. Re-check missingness


In [ ]:
##SOLUTION
df["age"].fillna(df["age"].median(), inplace=True)

df["embarked"].fillna(df["embarked"].mode()[0], inplace=True)

df["deck"] = df["deck"].cat.add_categories("Unknown")
df["deck"].fillna("Unknown", inplace=True)

df.isna().sum().sort_values(ascending=False).head(8)


<details>
<summary>💡 Why median/mode?</summary>

- **Age (numeric):** median is robust to outliers
- **Embarked (categorical):** mode reflects the most common category
- **Deck (string):** "Unknown" preserves the fact that a value should exist but wasn’t recorded
</details>


<details>
<summary>💡 Error when trying to fill deck?</summary>

The `deck` column is stored as a *Categorical dtype* in pandas.
To fill missing values with a new label (`"Unknown"`), you must first **add it to the category list**:
```python
df["deck"] = df["deck"].cat.add_categories("Unknown")
```

Alternatively, you can convert deck to string before filling if you don’t need the categorical type:
```python
df["deck"] = df["deck"].astype("string")
```
</details>


### Checkpoint 2

In [ ]:
assert df["age"].isna().sum() == 0
assert df["embarked"].isna().sum() == 0
assert df["deck"].isna().sum() == 0
print("✅")

## 3) Reshape

**Goal:**
You received a **summary sheet** from the museum team.
It shows passenger survival rates, but the layout is **wide** — each embarkation port is a column.
Your job: **convert this wide table back to tidy format**, where each row is one observation.

**Given:**
- A wide DataFrame called `messy`
- Columns: `class`, `sex`, `C`, `Q`, `S` → representing embarkation ports
- Each cell shows the **average survival rate** for that group

**Do this:**
1. Inspect the messy wide table (run the next cell)
2. Use `melt()` to make it tidy again with:
   - `id_vars=["class", "sex"]`
   - `var_name="embarked"`
   - `value_name="survival_rate"`


In [ ]:
messy = (
    df.groupby(["class", "sex", "embarked"], observed=True)["survived"]
      .mean()
      .round(3)
      .reset_index()
      .pivot(index=["class", "sex"], columns="embarked", values="survived")
      .reset_index()
)
messy.head()

In [ ]:
#SOLUTION
tidy = messy.melt(
    id_vars=["class","sex"],
    var_name="embarked",
    value_name="survival_rate"
)

tidy.head()


<details>
<summary>💡 Hint</summary>

- `melt()` reverses a pivot: it takes columns and turns them into rows.
- Use:
  ```python
  tidy = messy.melt(
      id_vars=["class", "sex"],
      var_name="embarked",
      value_name="survival_rate"
  )


In [ ]:
assert set(tidy.columns) == {"class","sex","embarked","survival_rate"}
print("✅")


## 4) Combine

**Goal:**
The museum’s data team discovered new passenger data and prepared **class descriptions** to enrich your dataset.

**Given:**
- A batch of **new rows**: **`new_data`**
- A prepared lookup table: **`class_info`**

**Do this:**
1. **Append** (`concat`) the enriched `new_Data` to `df`
2. Merge **`class_description`** from `class_info` into `df`



In [ ]:
new_data = sns.load_dataset("titanic").tail(100)

In [ ]:
#SOLUTION
df = pd.concat([df, new_data], axis=0, ignore_index=True)

<details>
<summary>💡 Hint (Concat)</summary>

- Use `pd.concat([df, new_rows], axis=0, ignore_index=True)` to append rows.
- `ignore_index=True` re-numbers the index 0..N-1.
</details>


In [ ]:
assert len(df) == 891
print("✅")

#### Merge prepared information

Now the data team gives you an additional **lookup table** named `class_info` that maps the Titanic’s passenger classes
(`First`, `Second`, `Third`) to **descriptive text** and **price category**.

We’ll merge it with `df` on the **'class'** column.


In [ ]:
prev_len = len(df)

class_info = pd.DataFrame({
    "class": ["First", "Second", "Third"],
    "class_description": [
        "First Class – luxury cabins and dining",
        "Second Class – comfortable standard cabins",
        "Third Class – basic accommodation and shared rooms"
    ],
    "price_category": ["high", "mid", "low"]
})

class_info

<details>
<summary>💡 Hint (Merge)</summary>

- Think of `merge()` as connecting tables **by a shared key** (here: <code>'class'</code>).
- Use `how='left'` so every passenger row is kept, even if the key isn't in the lookup.
- Syntax:
  <code>df.merge(class_info, on="class", how="left")</code>
- `validate="m:1"` checks that multiple passengers can map to one class record.
</details>


In [ ]:
df_merged = df.merge(
    class_info,
    on="class",
    how="left",
    validate="m:1"
)

df_merged[["class", "sex", "fare", "class_description", "price_category"]].head()

<details>
<summary>💡 Hint (Merge)</summary>

- Your **join keys must match in both *name* and *dtype*** on both sides.
- Since <code>df['fare_band']</code> is an **Interval** and your lookup uses a **string key** (<code>fare_band_key</code>), create a matching key in <code>df</code> with:<br>
  <code>df['fare_band_key'] = df['fare_band'].astype(str)</code>
- Then do a **left merge** on <code>fare_band_key</code> so you keep all rows from <code>df</code>.
- Use <code>validate="m:1"</code> to ensure many-to-one (many passengers → one label per band).
</details>


In [ ]:
assert len(df_merged) == prev_len, "❌ Row count changed, check merge keys."

missing_desc = df_merged["class_description"].isna().sum()
assert missing_desc == 0
print("✅")

